In [1]:
import pandas as pd
import numpy as np

# Import the first CSV file
fights_df = pd.read_csv('ufc_fights_data.csv')

# Import the second CSV file
fighters_df = pd.read_csv('ufc_fighters_data.csv')
pd.set_option('display.max_columns', None)
fighters_df.drop('url', axis=1, inplace=True) 


In [2]:
#clean fighters_df

# Replace '--' with NaN across the whole DataFrame
fighters_df.replace('--', np.nan, inplace=True)

# Remove leading colons
cols_to_clean = ['SLpM', 'Str. Acc.', 'SApM', 'Str. Def','TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']
for col in cols_to_clean:
  fighters_df[col] = fighters_df[col].astype(str).str.lstrip(':')

#clean weight, reach, and height
fighters_df['weight'] = fighters_df['weight'].astype(str).str.replace('lbs.', '', regex=False).str.strip()
fighters_df['weight'] = pd.to_numeric(fighters_df['weight'], errors='coerce')

fighters_df['reach'] = pd.to_numeric(
    fighters_df['reach'].astype(str).str.replace('"', '', regex=False).str.strip(),
    errors='coerce'
)

def parse_height(h):
    if isinstance(h, str) and "'" in h:
        ft, inch = h.split("'")
        return int(ft.strip()) * 12 + int(inch.strip().replace('"', ''))
    return None

fighters_df['height'] = fighters_df['height'].apply(parse_height)


#change percents to decimals
percent_cols = ['Str. Acc.', 'Str. Def', 'TD Acc.', 'TD Def.']
for col in percent_cols:
    fighters_df[col] = (
        fighters_df[col]
        .astype(str)
        .str.replace('%', '', regex=False)  # Remove %
        .str.strip()                        # Trim whitespace
        .astype(float) / 100                # Convert to decimal
    )


#format dob
fighters_df['dob'] = pd.to_datetime(fighters_df['dob'], errors='coerce')


#turn record collumn to wins, losses, draws, nc
fighters_df[['wins', 'losses', 'draws', 'nc']] = fighters_df['record'].str.extract(
    r'^(\d+)-(\d+)(?:-(\d+))?(?:\s*\((\d+)\s*NC\))?$'
)
cols = ['wins', 'losses', 'draws', 'nc']
fighters_df[cols] = fighters_df[cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

fighters_df.head(10)
fighters_df.to_csv('ufc_fighters_cleaned.csv', index=False)

In [3]:
#clean fights_df

#replace '--' values and turn percents to decimals
percent_cols = ['p1_SIG_STR_PCT', 'p2_SIG_STR_PCT', 'p1_TD_PCT', 'p2_TD_PCT', 'p1_R1_SIG_STR_PCT', 'p2_R1_SIG_STR_PCT', 'p1_R1_TD_PCT', 'p2_R1_TD_PCT', 'p1_SIG_STR_PCT_DETAILED', 'p2_SIG_STR_PCT_DETAILED', 'p1_R1_SIG_STR_PCT_DETAILED', 'p2_R1_SIG_STR_PCT_DETAILED']
for col in percent_cols:
    fights_df[col] = (
        fights_df[col]
        .astype(str)
        .replace({'--': np.nan, '---': np.nan})  # Handle missing values first
        .str.replace('%', '', regex=False)  # Remove percentage symbols
        .str.strip()  # Clean whitespace
        .astype(float) / 100  # Convert to decimal
    )


#format event date
fights_df['event_date'] = pd.to_datetime(fights_df['event_date'], errors='coerce')


# columns to split
cols_to_split = ['p1_SIG_STR', 'p2_SIG_STR', 'p1_TOTAL_STR', 'p2_TOTAL_STR', 'p1_TD', 'p2_TD', 'p1_R1_SIG_STR', 'p2_R1_SIG_STR', 'p1_R1_TOTAL_STR', 'p2_R1_TOTAL_STR', 'p1_R1_TD', 'p2_R1_TD', 'p1_SIG_STR_LANDED_ATTEMPTED', 'p2_SIG_STR_LANDED_ATTEMPTED', 'p1_HEAD_LANDED_ATTEMPTED', 'p2_HEAD_LANDED_ATTEMPTED', 'p1_BODY_LANDED_ATTEMPTED', 'p2_BODY_LANDED_ATTEMPTED', 'p1_LEG_LANDED_ATTEMPTED', 'p2_LEG_LANDED_ATTEMPTED', 'p1_DISTANCE_LANDED_ATTEMPTED', 'p2_DISTANCE_LANDED_ATTEMPTED', 'p1_CLINCH_LANDED_ATTEMPTED', 'p2_CLINCH_LANDED_ATTEMPTED', 'p1_GROUND_LANDED_ATTEMPTED', 'p2_GROUND_LANDED_ATTEMPTED', 'p1_R1_SIG_STR_LANDED_ATTEMPTED', 'p2_R1_SIG_STR_LANDED_ATTEMPTED', 'p1_R1_HEAD_LANDED_ATTEMPTED', 'p2_R1_HEAD_LANDED_ATTEMPTED', 'p1_R1_BODY_LANDED_ATTEMPTED', 'p2_R1_BODY_LANDED_ATTEMPTED', 'p1_R1_LEG_LANDED_ATTEMPTED', 'p2_R1_LEG_LANDED_ATTEMPTED', 'p1_R1_DISTANCE_LANDED_ATTEMPTED', 'p2_R1_DISTANCE_LANDED_ATTEMPTED', 'p1_R1_CLINCH_LANDED_ATTEMPTED', 'p2_R1_CLINCH_LANDED_ATTEMPTED', 'p1_R1_GROUND_LANDED_ATTEMPTED', 'p2_R1_GROUND_LANDED_ATTEMPTED']

#Loop over them, split into “_LANDED” and “_ATTEMPTED”, convert to ints
for col in cols_to_split:
    # decide new column names
    if col.endswith('_LANDED_ATTEMPTED'):
        base = col.rsplit('_', 2)[0]       # turns 'p1_HEAD_LANDED_ATTEMPTED' → 'p1_HEAD'
        landed_col    = f"{base}_LANDED"
        attempted_col = f"{base}_ATTEMPTED"
    else:
        # for columns like 'p1_SIG_STR'
        landed_col    = f"{col}_LANDED"
        attempted_col = f"{col}_ATTEMPTED"

    # split “63 of 129” → ['63','129'], then to numeric
    fights_df[[landed_col, attempted_col]] = (
        fights_df[col]
          .str.split(' of ', expand=True)
          .astype(int)
    )

# Drop the originals
fights_df.drop(columns=cols_to_split, inplace=True)


# time collumns needing reformatting
time_cols = ['p1_CTRL', 'p2_CTRL', 'p1_R1_CTRL', 'p2_R1_CTRL' ]

#Convert each to total seconds
for col in time_cols:
    # Replace non-time strings with NaN
    fights_df[col] = fights_df[col].replace({'--': np.nan, '---': np.nan})
    
    # Convert to "00:MM:SS" format then to seconds
    fights_df[col] = (
        pd.to_timedelta('00:' + fights_df[col].fillna('00:00'), errors='coerce')
        .dt.total_seconds()
        .astype('Int64')  # Allows integer NaNs
    )

fights_df.head(10)

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED
0,Mateusz Gamrot,Ludovit Klein,0,0,0.59,0.32,0.54,NaN,0,0,0,0,525,23,0,0,0.52,0.14,0.57,NaN,0,0,0,0,186,0,0.59,0.32,0.52,0.14,Decision - Unanimous,Herb Dean,1,2025-05-31,65,110,24,74,142,224,85,145,6,11,0,0,10,19,2,14,27,43,6,19,4,7,0,0,53,98,10,49,7,7,13,23,5,5,1,2,38,73,23,73,3,4,1,1,24,33,0,0,3,12,2,11,4,4,0,3,3,3,0,0,4,13,2,14,2,2,0,0,4,4,0,0
1,Billy Ray Goff,Ramiz Brahimaj,0,0,0.33,1.00,0.00,0.66,0,1,0,0,59,102,0,0,0.33,1.00,0.00,0.66,0,1,0,0,59,102,0.33,1.00,0.33,1.00,Submission,Chris Tognoni,0,2025-05-31,1,3,1,1,2,4,9,10,0,1,2,3,1,3,1,1,2,4,9,10,0,1,2,3,0,1,0,0,1,2,1,1,0,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,0,0,1,2,1,1,0,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0
2,Dustin Jacoby,Bruno Lopes,1,0,0.58,0.33,NaN,NaN,0,0,0,0,30,0,1,0,0.58,0.33,NaN,NaN,0,0,0,0,30,0,0.58,0.33,0.58,0.33,KO/TKO,Mike Beltran,1,2025-05-31,17,29,3,9,17,30,5,12,0,0,0,0,17,29,3,9,17,30,5,12,0,0,0,0,16,28,1,4,0,0,1,2,1,1,1,3,11,21,2,7,6,8,1,2,0,0,0,0,16,28,1,4,0,0,1,2,1,1,1,3,11,21,2,7,6,8,1,2,0,0,0,0
3,Ketlen Vieira,Macy Chiasson,0,0,0.43,0.51,0.00,0.00,0,0,0,0,606,33,0,0,0.37,0.54,NaN,NaN,0,0,0,0,66,8,0.43,0.51,0.37,0.54,Decision - Unanimous,Mark Smith,1,2025-05-31,28,64,22,43,106,174,52,77,0,1,0,4,18,48,17,31,29,64,18,32,0,0,0,0,25,60,8,25,3,3,6,9,0,1,8,9,15,47,20,39,5,6,2,4,8,11,0,0,15,44,5,16,3,3,5,7,0,1,7,8,13,43,17,29,5,5,0,2,0,0,0,0
4,Zachary Reese,Dusko Todorovic,0,0,0.46,0.53,0.66,0.30,0,1,0,1,79,330,0,0,0.54,0.66,1.00,0.75,0,1,0,1,66,181,0.46,0.53,0.54,0.66,Decision - Unanimous,Herb Dean,1,2025-05-31,50,108,35,65,73,135,73,109,2,3,4,13,6,11,10,15,19,27,31,38,2,2,3,4,21,72,22,50,20,23,8,10,9,13,5,5,43,98,22,50,7,10,6,6,0,0,7,9,3,7,8,12,3,4,1,2,0,0,1,1,2,6,2,6,4,5,1,1,0,0,7,8
5,Jafel Filho,Allan Nascimento,0,0,0.46,0.50,0.57,0.16,1,1,0,2,380,423,0,0,0.55,0.44,0.50,0.00,1,0,0,0,247,4,0.46,0.50,0.55,0.44,Decision - Unanimous,Chris Tognoni,0,2025-05-31,14,30,12,24,40,63,30,58,4,7,1,6,11,20,4,9,22,37,7,12,2,4,0,3,7,20,3,12,7,9,4,4,0,1,5,8,6,20,9,19,6,8,1,1,2,2,2,4,6,12,0,3,5,7,1,1,0,1,3,5,4,11,4,9,5,7,0,0,2,2,0,0
6,Kurt Holobaugh,Jordan Leavitt,0,0,0.00,1.00,NaN,0.75,0,1,0,0,0,87,0,0,0.00,1.00,NaN,0.75,0,1,0,0,0,87,0.00,1.00,0.00,1.00,Submission,Mike Beltran,0,2025-05-31,0,1,

In [4]:
# merge the two datasets

# First, merge p1 fighter stats
fights_df = fights_df.merge(
    fighters_df,
    left_on='p1_fighter',
    right_on='name',  # This matches p1_fighter with the name column in fighters_df
    how='left'
)
fights_df.drop(columns=['name'], inplace=True)

# Only include the specific columns you want to rename with p1_ prefix
p1_cols = ['height', 'weight', 'reach', 'stance', 'dob', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']
fights_df.rename(columns={col: f'p1_{col}' for col in p1_cols}, inplace=True)

# Then merge p2 fighter stats
fights_df = fights_df.merge(
    fighters_df,
    left_on='p2_fighter',
    right_on='name',  # This matches p2_fighter with the name column in fighters_df
    how='left'
)
fights_df.drop(columns=['name'], inplace=True)

# Use the same columns for p2 as we did for p1
p2_cols = ['height', 'weight', 'reach', 'stance', 'dob', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']
fights_df.rename(columns={col: f'p2_{col}' for col in p2_cols}, inplace=True)


# Drop unwanted columns
columns_to_drop = ['record_x', 'wins_x', 'losses_x', 'draws_x', 'nc_x', 'record_y', 'wins_y', 'losses_y', 'draws_y', 'nc_y']

# Check if columns exist before trying to drop them
existing_columns = [col for col in columns_to_drop if col in fights_df.columns]
if existing_columns:
    fights_df.drop(columns=existing_columns, inplace=True)

fights_df.head(10)

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_dob,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_dob,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.
0,Mateusz Gamrot,Ludovit Klein,0,0,0.59,0.32,0.54,NaN,0,0,0,0,525,23,0,0,0.52,0.14,0.57,NaN,0,0,0,0,186,0,0.59,0.32,0.52,0.14,Decision - Unanimous,Herb Dean,1,2025-05-31,65,110,24,74,142,224,85,145,6,11,0,0,10,19,2,14,27,43,6,19,4,7,0,0,53,98,10,49,7,7,13,23,5,5,1,2,38,73,23,73,3,4,1,1,24,33,0,0,3,12,2,11,4,4,0,3,3,3,0,0,4,13,2,14,2,2,0,0,4,4,0,0,70.0,155.0,70.0,Southpaw,1990-12-11,3.23,0.50,3.22,0.58,5.25,0.35,0.90,0.1,67.0,155.0,72.0,Southpaw,1995-02-22,3.81,0.54,3.22,0.54,1.70,0.51,0.91,0.0
1,Billy Ray Goff,Ramiz Brahimaj,0,0,0.33,1.00,0.00,0.66,0,1,0,0,59,102,0,0,0.33,1.00,0.00,0.66,0,1,0,0,59,102,0.33,1.00,0.33,1.00,Submission,Chris Tognoni,0,2025-05-31,1,3,1,1,2,4,9,10,0,1,2,3,1,3,1,1,2,4,9,10,0,1,2,3,0,1,0,0,1,2,1,1,0,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,0,0,1,2,1,1,0,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,70.0,170.0,72.0,Switch,1998-06-18,8.56,0.46,5.71,0.59,2.01,0.17,1.00,0.0,70.0,170.0,72.0,Orthodox,1992-11-17,2.26,0.44,4.04,0.43,1.51,0.35,0.44,1.5
2,Dustin Jacoby,Bruno Lopes,1,0,0.58,0.33,NaN,NaN,0,0,0,0,30,0,1,0,0.58,0.33,NaN,NaN,0,0,0,0,30,0,0.58,0.33,0.58,0.33,KO/TKO,Mike Beltran,1,2025-05-31,17,29,3,9,17,30,5,12,0,0,0,0,17,29,3,9,17,30,5,12,0,0,0,0,16,28,1,4,0,0,1,2,1,1,1,3,11,21,2,7,6,8,1,2,0,0,0,0,16,28,1,4,0,0,1,2,1,1,1,3,11,21,2,7,6,8,1,2,0,0,0,0,75.0,205.0,76.0,Orthodox,1988-04-04,5.33,0.47,4.05,0.57,0.33,0.25,0.62,0.0,74.0,205.0,74.0,Orthodox,1993-04-24,2.89,0.43,3.00,0.60,2.68,0.33,1.00,0.0
3,Ketlen Vieira,Macy Chiasson,0,0,0.43,0.51,0.00,0.00,0,0,0,0,606,33,0,0,0.37,0.54,NaN,NaN,0,0,0,0,66,8,0.43,0.51,0.37,0.54,Decision - Unanimous,Mark Smith,1,2025-05-31,28,64,22,43,106,174,52,77,0,1,0,4,18,48,17,31,29,64,18,32,0,0,0,0,25,60,8,25,3,3,6,9,0,1,8,9,15,47,20,39,5,6,2,4,8,11,0,0,15,44,5,16,3,3,5,7,0,1,7,8,13,43,17,29,5,5,0,2,0,0,0,0,68.0,135.0,68.0,Orthodox,1991-08-26,2.92,0.41,3.90,0.51,1.39,0.44,0.88,0.6,71.0,135.0,72.0,Orthodox,1991-07-27,3.77,0.46,2.79,0.44,2.37,0.38,0

In [5]:
fights_df.to_csv('ufc_cleaned.csv', index=False)